## Create all required methods

In [ ]:
library(data.table)

getDistro <- function(ldls, genotype, het =T){
  if (het){
    sampleZyg <- subset(genotype$sample, het == 1)
  }else{
    sampleZyg <- subset(genotype$sample, hom == 1)
  }
  sampleZyg$eid <- as.numeric(sampleZyg$eid)
  dataset <- merge( sampleZyg, ldls, by="eid", all.x=T)
  treated <- subset(dataset, llm_bl==1 )
  untreated <-  subset(dataset, llm_bl==0)
  ls <- list() 
  ls[[1]] <-  dataset$max_ldl
  ls[[2]] <-  treated$max_ldl
  ls[[3]] <-  untreated$max_ldl
  
  ls2 <- list() 
  ls2[[1]] <-  dataset$eid
  ls2[[2]] <-  treated$eid
  ls2[[3]] <-  untreated$eid
  return(list(LDL=ls, EID= ls2))
}

getsummaryLine <- function(values, variant){
  return(sprintf("%.2f(%.2f,%.2f,%.2f,%.2f)\nnind=%i,\nnvar=%i", median(values ,na.rm=T), summary(values,na.rm=T)[1], summary(values,na.rm=T)[2], summary(values,na.rm=T)[5], summary(values,na.rm=T)[6], length(values), variant))
}
getsummary <- function(distros, variant){
  return(list(getsummaryLine(distros[[1]],variant),
              getsummaryLine(distros[[2]],variant),
              getsummaryLine(distros[[3]],variant)))
} 

getData <- function(fileName){
  fileName <- paste(fileName,".participants", sep="")
  if(file.exists(paste(fileName,".bed", sep=""))){
    out.mat  <- snpStats::read.plink(paste(fileName, ".bed", sep=""), paste(fileName, ".bim", sep=""), paste(fileName, ".fam", sep=""))
    status_het   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==1)>=1,1,0) })
    gc()
    status_hom   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==0)>=1,1,0) })
    gc()
    
    eid <- rownames(out.mat$fam)
    ## calculate mean (IQR) count for Het and Hom
    genotype <- list(sample = data.table(eid = eid, 
                                        het = status_het , 
                                        hom = status_hom),
                      variant = nrow(out.mat$map))

    return(genotype)
  }else{
    return(NULL)
  }
}

## Load Data

In [ ]:
phenotype <- as.data.frame(fread("data_participant.tsv"))

In [ ]:
phenotype$max_ldl <- apply(phenotype[,c("p30780_i0", "p30780_i1")],1, FUN=function(x){ ifelse(length(na.omit(x))==0, NA, max(na.omit(x)))})
phenotype$llm_bl <- apply( phenotype[,c('p6177_i0','p6177_i1','p6177_i2','p6177_i3')], 1, FUN = function(x){
    ifelse(sum(grepl("Cholesterol lowering medication", x))>=1,1,0)
})

In [ ]:
pheno <- phenotype[,c("eid", "max_ldl", "llm_bl")]

## Genetic Analysis 
### LDLR
#### LDL-C Levels, variant and individuals

In [ ]:
files <- c("predicted.pathogenic","predicted.missense.pathogenic","predicted.nonmissense.pathogenic",
           "predicted.benign","predicted.missense.benign","predicted.nonmissense.benign",
           "predicted.VUS","predicted.missense.VUS","predicted.nonmissense.VUS",
           "clinvar.pathogenic","clinvar.missense.pathogenic","clinvar.nonmissense.pathogenic",
           "clinvar.benign","clinvar.missense.benign","clinvar.nonmissense.benign",
           "clinvar.VUS","clinvar.missense.VUS","clinvar.nonmissense.VUS",           
           "pathogenic","missense.pathogenic","nonmissense.pathogenic",
           "benign","missense.benign","nonmissense.benign",
           "VUS","missense.VUS","nonmissense.VUS")

In [ ]:
lsLDL<- list()
lsEID<- list()
lsName<- list()
for (f in files){
        data1 <- getData(paste("genes/Chr19/bed/", f, sep=""))
        distros <- getDistro(pheno, data1, het=T )
        lsName[[ length(lsName) + 1 ]] <- f
        lsLDL[[ length(lsLDL) + 1 ]] <- distros$LDL
        lsEID[[ length(lsEID) + 1 ]] <- distros$EID
        print(f)
        print('overall')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[1]],data1$variant))
        print('treated')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[2]],data1$variant))
        print('untreated')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[3]],data1$variant))
        cat("\n\n")
}

#### Statistical Test: Difference LDL Predicted vs Clinvar

In [ ]:

showStats <- function(tbl){
    for (row in 1:nrow(tbl)){
        cat(paste(files[tbl[row,"group1"]], " vs ", files[tbl[row,"group2"]]))
        for (treatmentGroup in 1:3){
            cat(sprintf("\n%s\n",treatmentGroup))
            print(round(median(lsLDL[[tbl[row,"group1"]]][[treatmentGroup]],na.rm=T)-
                        median(lsLDL[[tbl[row,"group2"]]][[treatmentGroup]],na.rm=T),2))
            print(wilcox.test(lsLDL[[tbl[row,"group1"]]][[treatmentGroup]],
                        lsLDL[[tbl[row,"group2"]]][[treatmentGroup]]))
        }
    }
}
tbl_predvsclinvar <- data.frame(group1=c(1,2,3,4,5,6,7,8,9), 
                                group2=c(10,11,12,13,14,15,16,17,18))
tbl_pathogenicvsbenign <- data.frame(group1=c(10,11,12,1,2,3,19,20,21), 
                                     group2=c(13,14,15,4,5,6,22,23,24))


showStats( tbl_predvsclinvar )
showStats( tbl_pathogenicvsbenign)

#### Prevalence in LDLR gene

In [ ]:
paste("prevalence is 1:", round(nrow(pheno)/length(unique(c(unlist(lsEID[[19]]))))))

#### Plot 

In [ ]:
par(mfrow=c(2,2))
# Overall 
plot(x=c(), y=c(), xlim=c(0,10), ylim=c(0,0.6), xlab="LDL-C", ylab="freq", main="Overall LDL-C levels")
lines(density(lsLDL[[10]][[1]], na.rm=T), col="red")
lines(density(lsLDL[[1]][[1]], na.rm=T), col="pink")

lines(density(lsLDL[[16]][[1]], na.rm=T), col="orange")
lines(density(lsLDL[[7]][[1]], na.rm=T), col="yellow")

lines(density(lsLDL[[13]][[1]], na.rm=T), col="green")
lines(density(lsLDL[[4]][[1]], na.rm=T), col="lightgreen")

plot.new()
legend("center",c("ClinVar Pathogenic",
                  "Predicted Pathogenic",
                  "ClinVar VUS",
                  "Predicted VUS",
                  "ClinVar Benign",
                  "Predicted Benign"),
                col=c("red",
                      "pink",
                      "orange",
                      "yellow",
                      "green",
                      "lightgreen"), lty=1)


# Treated
plot(x=c(), y=c(), xlim=c(0,10), ylim=c(0,0.8), xlab="LDL-C", ylab="freq", main="Treated LDL-C levels")
lines(density(lsLDL[[10]][[2]], na.rm=T), col="red")
lines(density(lsLDL[[1]][[2]], na.rm=T), col="pink")

lines(density(lsLDL[[16]][[2]], na.rm=T), col="orange")
lines(density(lsLDL[[7]][[2]], na.rm=T), col="yellow")

lines(density(lsLDL[[13]][[2]], na.rm=T), col="green")
lines(density(lsLDL[[4]][[2]], na.rm=T), col="lightgreen")


# Untreated
plot(x=c(), y=c(), xlim=c(0,10), ylim=c(0,0.8), xlab="LDL-C", ylab="freq", main="Untreated LDL-C levels")
lines(density(lsLDL[[10]][[3]], na.rm=T), col="red")
lines(density(lsLDL[[1]][[3]], na.rm=T), col="pink")

lines(density(lsLDL[[16]][[3]], na.rm=T), col="orange")
lines(density(lsLDL[[7]][[3]], na.rm=T), col="yellow")

lines(density(lsLDL[[13]][[3]], na.rm=T), col="green")
lines(density(lsLDL[[4]][[3]], na.rm=T), col="lightgreen")


#### Test Homozygous & compound HeFH

In [ ]:
fileName  <- paste("genes/Chr19/bed/", "pathogenic.participants", sep="")
out.mat  <- snpStats::read.plink(paste(fileName, ".bed", sep=""), paste(fileName, ".bim", sep=""), paste(fileName, ".fam", sep=""))
status_het   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==1)>=1,1,0) })
status_hom   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==0)==1,1,0) })
status_compound_heFH   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==1)>1,1,0) })

In [ ]:
sum(status_het)
sum(status_hom)
sum(status_compound_heFH )

In [ ]:
# compunds are
lapply(which(status_compound_heFH==1), FUN = function(eid){
    colnames(out.mat$genotypes)[which(as(out.mat$genotypes[eid,],"numeric")==1)]
})

In [ ]:
# to retrieve the treatment and LDL
subset(pheno, eid %in% c("1125055","1180862","3204510","3380907","5468065"))[,c("max_ldl","llm_bl")]

### PCSK9

In [ ]:
files_pcsk9 <- c("clinvar.pathogenic","clinvar.benign","clinvar.VUS")

In [ ]:
lsLDL<- list()
lsEID<- list()
lsName<- list()
for (f in files_pcsk9){
        data1 <- getData(paste("genes/Chr1/bed/", f, sep=""))
        distros <- getDistro(pheno, data1, het=T )
        lsName[[ length(lsName) + 1 ]] <- f
        lsLDL[[ length(lsLDL) + 1 ]] <- distros$LDL
        lsEID[[ length(lsEID) + 1 ]] <- distros$EID
        print(f)
        print('overall')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[1]],data1$variant))
        print('treated')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[2]],data1$variant))
        print('untreated')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[3]],data1$variant))
        cat("\n\n")
}

#### Statistical Test

In [ ]:

showStats <- function(tbl){
    for (row in 1:nrow(tbl)){
        cat(paste(files_pcsk9[tbl[row,"group1"]], " vs ", files_pcsk9[tbl[row,"group2"]]))
        for (treatmentGroup in 1:3){
            cat(sprintf("\n%s\n",treatmentGroup))
            print(round(median(lsLDL[[tbl[row,"group1"]]][[treatmentGroup]],na.rm=T)-
                        median(lsLDL[[tbl[row,"group2"]]][[treatmentGroup]],na.rm=T),2))
            print(wilcox.test(lsLDL[[tbl[row,"group1"]]][[treatmentGroup]],
                        lsLDL[[tbl[row,"group2"]]][[treatmentGroup]]))
        }
    }
}
tbl_pathogenicvsbenign <- data.frame(group1=c(1), 
                                     group2=c(2))


showStats( tbl_pathogenicvsbenign)

#### Test Homozygous & compound HeFH

In [ ]:
fileName  <- paste("genes/Chr1/bed/", "clinvar.pathogenic.participants", sep="")
out.mat  <- snpStats::read.plink(paste(fileName, ".bed", sep=""), paste(fileName, ".bim", sep=""), paste(fileName, ".fam", sep=""))
status_het   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==1)>=1,1,0) })
status_hom   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==0)==1,1,0) })
status_compound_heFH   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==1)>1,1,0) })

In [ ]:
sum(status_het)
sum(status_hom)
sum(status_compound_heFH )

### APOB

In [ ]:
files_apob <- c("clinvar.pathogenic","clinvar.benign","clinvar.VUS")

In [ ]:
lsLDL<- list()
lsEID<- list()
lsName<- list()
for (f in files_apob){
        data1 <- getData(paste("genes/Chr2/bed/", f, sep=""))
        distros <- getDistro(pheno, data1, het=T )
        lsName[[ length(lsName) + 1 ]] <- f
        lsLDL[[ length(lsLDL) + 1 ]] <- distros$LDL
        lsEID[[ length(lsEID) + 1 ]] <- distros$EID
        print(f)
        print('overall')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[1]],data1$variant))
        print('treated')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[2]],data1$variant))
        print('untreated')
        print(getsummaryLine(lsLDL[[length(lsLDL)]][[3]],data1$variant))
        cat("\n\n")
}

#### Statistical Test

In [ ]:

showStats <- function(tbl){
    for (row in 1:nrow(tbl)){
        cat(paste(files_apob[tbl[row,"group1"]], " vs ", files_apob[tbl[row,"group2"]]))
        for (treatmentGroup in 1:3){
            cat(sprintf("\n%s\n",treatmentGroup))
            print(round(median(lsLDL[[tbl[row,"group1"]]][[treatmentGroup]],na.rm=T)-
                        median(lsLDL[[tbl[row,"group2"]]][[treatmentGroup]],na.rm=T),2))
            print(wilcox.test(lsLDL[[tbl[row,"group1"]]][[treatmentGroup]],
                        lsLDL[[tbl[row,"group2"]]][[treatmentGroup]]))
        }
    }
}
tbl_pathogenicvsbenign <- data.frame(group1=c(1), 
                                     group2=c(2))


showStats( tbl_pathogenicvsbenign)

#### Test Homozygous & compound HeFH

In [ ]:
fileName  <- paste("genes/Chr2/bed/", "clinvar.pathogenic.participants", sep="")
out.mat  <- snpStats::read.plink(paste(fileName, ".bed", sep=""), paste(fileName, ".bim", sep=""), paste(fileName, ".fam", sep=""))
status_het   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==1)>=1,1,0) })
status_hom   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==0)==1,1,0) })
status_compound_heFH   <-  apply(as(out.mat$genotypes[rownames(out.mat$fam), ], 'numeric'), 1, FUN = function(x){ ifelse(sum(na.omit(x)==1)>1,1,0) })

In [ ]:
sum(status_het)
sum(status_hom)
sum(status_compound_heFH )

### Double HEFH

In [ ]:
eid_ldlr <- unique(unlist(getDistro(pheno, getData("genes/Chr19/bed/pathogenic"), het=T )$EID))
eid_pcsk9 <- unique(unlist(getDistro(pheno, getData("genes/Chr1/bed/clinvar.pathogenic"), het=T )$EID))
eid_apob <- unique(unlist(getDistro(pheno, getData("genes/Chr2/bed/clinvar.pathogenic"), het=T )$EID))

In [ ]:
length(eid_apob)
length(eid_pcsk9)
length(eid_ldlr)

In [ ]:
sum(eid_ldlr %in% eid_pcsk9)
sum(eid_apob %in% eid_pcsk9)
sum(eid_apob %in% eid_ldlr)

### Total Prevalence 

In [ ]:
pathoPatient <- unique(c(eid_ldlr,eid_apob , eid_pcsk9))
paste("prevalence is 1:", round(nrow(pheno)/length(pathoPatient )))

In [ ]:
length(pathoPatient )

### Create Outcome files


In [ ]:
eid_ldlr <-  unique(unlist(getDistro(pheno, getData("genes/Chr19/pathogenic"), het=T )$EID))
eid_pcsk9 <-  unique(unlist(getDistro(pheno, getData("genes/Chr1/clinvar.pathogenic"), het=T )$EID))
eid_apob <-  unique(unlist(getDistro(pheno, getData("genes/Chr2/clinvar.pathogenic"), het=T )$EID))

In [ ]:
alleid_w450k <- read.table("genes/Chr19/pathogenic.participants.fam")

In [ ]:
outcome_df <- data.frame(eid=alleid_w450k$V1)
head(outcome_df )

In [ ]:
outcome_df[,"LDLR"] <- ifelse(outcome_df$eid %in% eid_ldlr,1,0)
outcome_df[,"APOB"] <- ifelse(outcome_df$eid %in% eid_apob,1,0)
outcome_df[,"PCSK9"] <- ifelse(outcome_df$eid %in% eid_pcsk9,1,0)
outcome_df[,"ANY"] <- apply(outcome_df[,2:4],1, FUN=function(x){ifelse(any(x==1),1,0)})

In [ ]:
head(outcome_df)
apply(outcome_df[,2:5],2, sum)

In [ ]:
saveRDS(outcome_df,"outcome.rds")

In [ ]:
readRDS("outcome.rds")